In [ ]:
# Import necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve,auc
from sklearn.model_selection import train_test_split,cross_val_predict
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from lightgbm import early_stopping
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed
import warnings

data=pd.read_csv('/home/tisinr/Dev/models/classifier/dataset/bank.csv',header=0, sep=';')
# Display the first few rows of the dataset
print(data.head())


In [ ]:
def rename_n_change(data):
    # Rename columns for better readability
    data.rename(columns={
        'marital':'marital_status',
        'default':'credit_default',
        'housing':'housing_loan',
        'loan':'personal_loan',
        'y':'response'}, inplace=True)
    #change data types
    data['response'] = data['response'].astype('category')
    data['marital_status'] = data['marital_status'].astype('category')
    data['education'] = data['education'].astype('category')
    data['job'] = data['job'].astype('category')
    data['contact'] = data['contact'].astype('category')
    data['month'] = data['month'].astype('category')
    data['day'] = data['day'].astype('category')
    data['credit_default'] = data['credit_default'].astype('category')
    data['housing_loan'] = data['housing_loan'].astype('category')
    data['personal_loan'] = data['personal_loan'].astype('category')
    return data
data=rename_n_change(data)


In [ ]:
X=data.drop(columns=['response'])
y=data['response']

In [ ]:
## Apply preprocessing steps to the dataset
# Create a pipeline for preprocessing
numeric_features = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
categorical_features = ['job', 'marital_status', 'education', 'month', 'housing_loan', 'personal_loan', 'credit_default']
# Create a column transformer to apply different preprocessing steps to different columns
pre_processor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)
# Create a pipeline that first applies the preprocessor and then fits a classifier
pre_pipeline = Pipeline(steps=[
    ('preprocessor', pre_processor)
])


In [ ]:
#Split the data into training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,stratify=y,random_state=78)

# Label encode the response variable
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
# Fit preprocessing on training data and transform both sets
X_train = pre_pipeline.fit_transform(X_train)
X_test = pre_pipeline.transform(X_test)  # No fitting on test data!

# Define models
#models = [
 #   ('Logistic Regression', LogisticRegression()),
 #   ('Decision Tree', DecisionTreeClassifier()),
  #  ('Random Forest', RandomForestClassifier(n_estimators=100)),
   # ('Gradient Boosting', GradientBoostingClassifier()),
  #  ('Support Vector', SVC(probability=True)),
  #  ('Gaussian Naive Bayes', GaussianNB()),
   # ('K Neighbors', KNeighborsClassifier())]


In [ ]:
smote = SMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
#Logistic Regression
logreg=LogisticRegression(class_weight= 'balanced')
logreg.fit(X_train,y_train)

In [ ]:
#Decision Tree Classifier
dtree=DecisionTreeClassifier()
dtree.fit(X_train,y_train)

In [ ]:
#Random Forest Classifier
rforest=RandomForestClassifier(class_weight= 'balanced',n_estimators=100)
rforest.fit(X_train,y_train)

In [ ]:
#Gradient Boosting Classifier
gbm=GradientBoostingClassifier()
gbm.fit(X_train,y_train)

In [ ]:
#Support Vector Classifier
svc=SVC(class_weight= 'balanced',probability=True)
svc.fit(X_train,y_train)

In [ ]:
#Gaussian Naive Bayes
gnb=GaussianNB()
gnb.fit(X_train,y_train)

In [ ]:
#K Neighbors Classifier
knn=KNeighborsClassifier()
knn.fit(X_train,y_train)

In [ ]:
# Initialize optimized models
xgb = XGBClassifier(
    n_estimators=100,  # Allow more rounds, rely on early stopping
    max_depth=3,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=-1,
    random_state=42,
    eval_metric='logloss',
    early_stopping_rounds=10
)
# Fit models with early stopping
xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
y_pred=xgb.predict(X_test)

# Feature selection based on XGBoost importance
selector = SelectFromModel(xgb, prefit=True)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)



In [ ]:
lgb = LGBMClassifier(
    n_estimators=100,
    max_depth=3,
    class_weight='balanced',
    num_leaves=15,  # Smaller for speed
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=-1,
    random_state=42,
    early_stopping_rounds=10,
    verbose=-1
)
lgb.fit(X_train, y_train, eval_set=[(X_test, y_test)])


In [ ]:
# Update models list with fitted models
models = [
    ('XGBoost', xgb),
    ('Lightgbm', lgb)
    ('Logistic Regression',logreg),
    ('Decision Tree',dtree),
    ('Random Forest',rforest),
    ('Gradient Boosting',gbm),
    ('SVC',svc), 
    ('Gaussian Naive Bayes',gnb),
    ('K Neighbors',knn)
]

In [ ]:
def plot_all_models_roc(models):

    plt.figure(figsize=(10, 8))
    
    # Plot diagonal line
    plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
    
    # Iterate through models
    for model_name, model in models:
        
        # Get predicted probabilities
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        
        # Calculate ROC curve and AUC
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        roc_auc = auc(fpr, tpr)
        
        # Plot ROC curve
        plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})')
    
    # Customize plot
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic - Model Comparison')
        plt.legend(loc='lower right')
        plt.grid(True)
    
    # Save plot
        plt.savefig('roc_comparison.png')
        #plt.close()
    return None


In [ ]:

plot_all_models_roc(models)